In [2]:
import nltk
from nltk import *
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag
nltk.download('twitter_samples')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')


tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

from nltk.corpus import stopwords
stop_words = stopwords.words('english')


[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\a670822\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\a670822\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\a670822\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\a670822\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\a670822\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))


In [14]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)
from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [15]:
import random

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
print(train_data)
test_data = dataset[7000:]

[({'#fridaydownpour': True, 'hope': True, 'clear': True, 'wkend': True, 'good': True, 'one': True, ':)': True}, 'Positive'), ({'say': True, ':-(': True}, 'Negative'), ({'someone': True, 'sc': True, ':(': True}, 'Negative'), ({'welcome': True, ':)': True}, 'Positive'), ({'mirror': True, 'beautiful': True, ':(': True}, 'Negative'), ({'guys': True, 'add': True, 'kik': True, 'mune': True, '874': True, '#kik': True, '#kikme': True, '#tagsforlikes': True, '#snapme': True, '#bored': True, '#hannibal': True, '#kikgirl': True, ':(': True}, 'Negative'), ({':(': True}, 'Negative'), ({'full': True, 'shaved': True, ':(': True}, 'Negative'), ({'caught': True, 'first': True, 'salmon': True, 'today': True, ':d': True}, 'Positive'), ({'1': True, 'daughter': True, '4yrs': True, 'old-porridge': True, 'fiend': True, ':-)': True}, 'Positive'), ({'sad': True, ':(': True}, 'Negative'), ({'ha': True, ':-)': True, 'guess': True, 'polite': True}, 'Positive'), ({':(': True, 'wish': True, 'could': True, 'say': Tr

In [17]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.9943333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2051.9 : 1.0
                     sad = True           Negati : Positi =     24.5 : 1.0
                follower = True           Positi : Negati =     23.4 : 1.0
                     bam = True           Positi : Negati =     22.8 : 1.0
                    glad = True           Positi : Negati =     18.7 : 1.0
              appreciate = True           Positi : Negati =     14.6 : 1.0
                followed = True           Negati : Positi =     13.9 : 1.0
                       😭 = True           Negati : Positi =     11.4 : 1.0
                    blog = True           Positi : Negati =     11.2 : 1.0
              definitely = True           Positi : Negati =     11.2 : 1.0
None


In [26]:
from nltk.tokenize import word_tokenize

custom_tweet = "god god meh sorrow sorrow nice sorrow"

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Positive


In [27]:
custom_tweet = "Congrats #SportStar on your 7th best goal from last season winning goal of the year :) #Baller #Topbin #oneofmanyworldies"
custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Positive
